# Dependencias

In [ ]:
!pip install keplergl
!pip install pymysql
!pip3 install pickle5

     |████████████████████████████████| 9.7 MB 9.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 47.9 MB/s 
     |████████████████████████████████| 6.3 MB 47.1 MB/s 
     |████████████████████████████████| 16.7 MB 50.6 MB/s 
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=bf6ab683bf6ef621a23455e7d01b6c0559b57fa95db65ac4737fcdca5aa2f086
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 256 kB 25.0 MB/s 


In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine
from datetime import datetime 
from functools import reduce
import numpy as np
from keplergl import KeplerGl
import pickle5 as pickle

# Conexión a la base de datos

In [ ]:
with open("cred_uber.json", "r") as f:
  cred=json.load(f)

In [ ]:
host = cred['host']
port = cred['port']
user = cred['username']
pwd = cred['password']
db = cred['db']

In [ ]:
flavor = "mysql"
client = "pymysql"

In [ ]:
conection=create_engine(f'{flavor}+{client}://{user}:{pwd}@{host}:{port}/{db}').connect()

## Lectura de tablas

In [ ]:
pd.read_sql(con=conection, sql='show tables')

,Tables_in_uber
0,apr
1,aug
2,bases
3,jul
4,jun
5,may
6,sep
7,stats


### Bases

In [ ]:
bases=pd.read_sql(con=conection, sql='bases')
bases

,Base Code,Base Name
0,B02512,Unter
1,B02598,Hinter
2,B02617,Weiter
3,B02682,Schmecken
4,B02764,Danach-NY
5,B02765,Grun
6,B02835,Dreist
7,B02836,Drinnen


### Estadísticas por día de cada base

In [ ]:
stats=pd.read_sql(con=conection, sql='stats')
stats['date']=pd.to_datetime(stats['date'])

In [ ]:
stats

,dispatching_base_number,date,active_vehicles,trips
0,B02512,2015-01-01,190.0,1132.0
1,B02765,2015-01-01,225.0,1765.0
2,B02764,2015-01-01,3427.0,29421.0
3,B02682,2015-01-01,945.0,7679.0
4,B02617,2015-01-01,1228.0,9537.0
...,...,...,...,...
349,B02764,2015-02-28,3952.0,39812.0
350,B02617,2015-02-28,1372.0,14022.0
351,B02682,2015-02-28,1386.0,14472.0
352,B02512,2015-02-28,230.0,1803.0


In [ ]:
stats['date'].min(),stats['date'].max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2015-02-28 00:00:00'))

### Tablas mensuales

In [ ]:
april=pd.read_sql(con=conection, sql='SELECT * from apr limit 100')
april

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
95,4/1/2014 6:54:00,40.7653,-73.9723,B02512
96,4/1/2014 6:56:00,40.7644,-73.9769,B02512
97,4/1/2014 6:57:00,40.7726,-73.9532,B02512
98,4/1/2014 6:59:00,40.7898,-73.9661,B02512


In [ ]:
may=pd.read_sql(con=conection, sql='SELECT * from may limit 100')
may

,Date/Time,Lat,Lon,Base
0,5/1/2014 0:02:00,40.7521,-73.9914,B02512
1,5/1/2014 0:06:00,40.6965,-73.9715,B02512
2,5/1/2014 0:15:00,40.7464,-73.9838,B02512
3,5/1/2014 0:17:00,40.7463,-74.0011,B02512
4,5/1/2014 0:17:00,40.7594,-73.9734,B02512
...,...,...,...,...
95,5/1/2014 6:07:00,40.7204,-74.0085,B02512
96,5/1/2014 6:07:00,40.7175,-74.0022,B02512
97,5/1/2014 6:07:00,40.7321,-73.9885,B02512
98,5/1/2014 6:08:00,40.7273,-73.9922,B02512


In [ ]:
june=pd.read_sql(con=conection, sql='SELECT * from jun limit 100')
june

,Date/Time,Lat,Lon,Base
0,6/1/2014 0:00:00,40.7293,-73.9920,B02512
1,6/1/2014 0:01:00,40.7131,-74.0097,B02512
2,6/1/2014 0:04:00,40.3461,-74.6610,B02512
3,6/1/2014 0:04:00,40.7555,-73.9833,B02512
4,6/1/2014 0:07:00,40.6880,-74.1831,B02512
...,...,...,...,...
95,6/1/2014 6:35:00,40.7543,-73.9817,B02512
96,6/1/2014 6:37:00,40.7751,-73.9633,B02512
97,6/1/2014 6:46:00,40.6952,-74.1784,B02512
98,6/1/2014 6:51:00,40.7621,-73.9817,B02512


### Catálogo de meses

In [ ]:
month=['apr', 
       'may',
       'jun',
       'jul',
       'aug', 
       'sep'
       ]

In [ ]:

ub=reduce(lambda x,y: x.merge(y, how='outer'), 
          list(map(
                    lambda x: pd.read_sql(con=conection, sql=f'{x}'), 
                  month)))

In [ ]:
ub.to_pickle('uber_db.pkl')

In [ ]:
conection.close()

# Transformación de los datos

In [ ]:
ub=pd.read_pickle('uber_db.pkl')

In [ ]:
ub

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [ ]:
ub.columns=['date_time', 'lat', 'lon', 'base']

In [ ]:
ub['date_time']=pd.to_datetime(ub['date_time'],errors='coerce')

## Cambiando claves por nombres de las bases

In [ ]:
d_bases=dict(zip(bases['Base Code'], bases['Base Name']))

In [ ]:
d_bases

{'B02512': 'Unter',
 'B02598': 'Hinter',
 'B02617': 'Weiter',
 'B02682': 'Schmecken',
 'B02764': 'Danach-NY',
 'B02765': 'Grun',
 'B02835': 'Dreist',
 'B02836': 'Drinnen'}

In [ ]:
ub['base']=ub['base'].map(d_bases)

In [ ]:
ub

,date_time,lat,lon,base
0,2014-04-01 00:11:00,40.7690,-73.9549,Unter
1,2014-04-01 00:17:00,40.7267,-74.0345,Unter
2,2014-04-01 00:21:00,40.7316,-73.9873,Unter
3,2014-04-01 00:28:00,40.7588,-73.9776,Unter
4,2014-04-01 00:33:00,40.7594,-73.9722,Unter
...,...,...,...,...
4534322,2014-09-30 22:57:00,40.7668,-73.9845,Danach-NY
4534323,2014-09-30 22:57:00,40.6911,-74.1773,Danach-NY
4534324,2014-09-30 22:58:00,40.8519,-73.9319,Danach-NY
4534325,2014-09-30 22:58:00,40.7081,-74.0066,Danach-NY


## Tipos de datos

In [ ]:
ub.dtypes

date_time    datetime64[ns]
lat                 float64
lon                 float64
base                 object
dtype: object

## Verificando si hay valores usentes

In [ ]:
ub.dropna().shape

(4534327, 4)

# Tabla de hechos

**Hechos:**
1) Viaje

**Dimensiones:** 
1) Latitud

2) Longitud 

3) Base 

4) Fecha

In [ ]:
ub['month']=pd.to_datetime(ub['date_time']).dt.strftime('%B-%Y')

In [ ]:
ub['frequency']=np.full(ub.shape[0],1)

In [ ]:
ub

,date_time,lat,lon,base,month,frequency
0,2014-04-01 00:11:00,40.7690,-73.9549,Unter,April-2014,1
1,2014-04-01 00:17:00,40.7267,-74.0345,Unter,April-2014,1
2,2014-04-01 00:21:00,40.7316,-73.9873,Unter,April-2014,1
3,2014-04-01 00:28:00,40.7588,-73.9776,Unter,April-2014,1
4,2014-04-01 00:33:00,40.7594,-73.9722,Unter,April-2014,1
...,...,...,...,...,...,...
4534322,2014-09-30 22:57:00,40.7668,-73.9845,Danach-NY,September-2014,1
4534323,2014-09-30 22:57:00,40.6911,-74.1773,Danach-NY,September-2014,1
4534324,2014-09-30 22:58:00,40.8519,-73.9319,Danach-NY,September-2014,1
4534325,2014-09-30 22:58:00,40.7081,-74.0066,Danach-NY,September-2014,1


In [ ]:
ub.dtypes

date_time    datetime64[ns]
lat                 float64
lon                 float64
base                 object
month                object
frequency             int64
dtype: object

In [ ]:
ub_g=ub.drop(['lat','lon'],axis=1).groupby(['base','month'],as_index=False).sum()

## Cubo sin datos geográficos

In [ ]:
ub_g.sort_values(by=['month','frequency'], ascending=False, ignore_index=True)

,base,month,frequency
0,Weiter,September-2014,377695
1,Hinter,September-2014,240600
2,Schmecken,September-2014,197138
3,Danach-NY,September-2014,178333
4,Unter,September-2014,34370
5,Hinter,May-2014,260549
6,Schmecken,May-2014,222883
7,Weiter,May-2014,122734
8,Unter,May-2014,36765
9,Danach-NY,May-2014,9504


In [ ]:
ub_g.dtypes

base         object
month        object
frequency     int64
dtype: object

## Tabla con latitud y longitud

In [ ]:
ub['hour']=ub['date_time'].map(lambda x: x.hour)

In [ ]:
ub['date']=ub['date_time'].dt.date

In [ ]:
aux=ub.drop(['date_time'],axis=1)

In [ ]:
aux

,lat,lon,base,month,frequency,hour,date
0,40.7690,-73.9549,Unter,April-2014,1,0,2014-04-01
1,40.7267,-74.0345,Unter,April-2014,1,0,2014-04-01
2,40.7316,-73.9873,Unter,April-2014,1,0,2014-04-01
3,40.7588,-73.9776,Unter,April-2014,1,0,2014-04-01
4,40.7594,-73.9722,Unter,April-2014,1,0,2014-04-01
...,...,...,...,...,...,...,...
4534322,40.7668,-73.9845,Danach-NY,September-2014,1,22,2014-09-30
4534323,40.6911,-74.1773,Danach-NY,September-2014,1,22,2014-09-30
4534324,40.8519,-73.9319,Danach-NY,September-2014,1,22,2014-09-30
4534325,40.7081,-74.0066,Danach-NY,September-2014,1,22,2014-09-30


In [ ]:
aux.dtypes

lat          float64
lon          float64
base          object
month         object
frequency      int64
hour           int64
date          object
dtype: object

In [ ]:
mapa=KeplerGl(height=400, witdh=600)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
aux=aux[['lat','lon','base','hour','date']]
#aux['date']=aux['date'].map(lambda x: x.strftime('%Y-%m-%d'))
aux['day']=pd.to_datetime(ub['date']).dt.strftime("%A")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Cubo con datos geográficos

In [ ]:
aux

,lat,lon,base,hour,date,day
0,40.7690,-73.9549,Unter,0,2014-04-01,Tuesday
1,40.7267,-74.0345,Unter,0,2014-04-01,Tuesday
2,40.7316,-73.9873,Unter,0,2014-04-01,Tuesday
3,40.7588,-73.9776,Unter,0,2014-04-01,Tuesday
4,40.7594,-73.9722,Unter,0,2014-04-01,Tuesday
...,...,...,...,...,...,...
4534322,40.7668,-73.9845,Danach-NY,22,2014-09-30,Tuesday
4534323,40.6911,-74.1773,Danach-NY,22,2014-09-30,Tuesday
4534324,40.8519,-73.9319,Danach-NY,22,2014-09-30,Tuesday
4534325,40.7081,-74.0066,Danach-NY,22,2014-09-30,Tuesday


## Exportando los cubos

In [ ]:
aux.to_csv('cubo_geo.csv')

In [ ]:
ub_g.to_csv('cubo_s_geo.csv')

In [ ]:
stats['dispatching_base_number']=stats['dispatching_base_number'].map(d_bases)
stats.to_csv('stats.csv')

In [ ]:
mapa.add_data(data=aux)

In [ ]:
mapa

## TD Latitud

In [ ]:
aux['lat']=aux['lat'].map(lambda x: round(x,6))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
td_latitude=pd.DataFrame({"lat_id":range(0,len(aux["lat"].unique())),"lat":aux["lat"].unique()})
td_latitude

,lat_id,lat
0,0,40.7690
1,1,40.7267
2,2,40.7316
3,3,40.7588
4,4,40.7594
...,...,...
7087,7087,41.1899
7088,7088,41.3180
7089,7089,40.4482
7090,7090,40.5571


## TD Longitud

In [ ]:
aux['lon']=aux['lon'].map(lambda x: round(x,6))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
td_longitude=pd.DataFrame({"lon_id":range(0,len(aux["lon"].unique())),"lon":aux["lon"].unique()})
td_longitude

,lon_id,lon
0,0,-73.9549
1,1,-74.0345
2,2,-73.9873
3,3,-73.9776
4,4,-73.9722
...,...,...
11448,11448,-73.3502
11449,11449,-74.3220
11450,11450,-74.5276
11451,11451,-73.4325


## TD Bases

In [ ]:
td_bases = pd.DataFrame(bases)
td_bases.rename(columns = {'Base Code':'base_code', 'Base Name':'base_name'}, inplace = True)
td_bases

,base_code,base_name
0,B02512,Unter
1,B02598,Hinter
2,B02617,Weiter
3,B02682,Schmecken
4,B02764,Danach-NY
5,B02765,Grun
6,B02835,Dreist
7,B02836,Drinnen


## TD Fecha

In [ ]:
td_dates = pd.DataFrame({'date_id':pd.to_datetime(aux['date'].drop_duplicates()).dt.strftime("%Y%m%d")})
td_dates['date'] = pd.to_datetime(td_dates["date_id"])
td_dates['name'] = td_dates["date"].dt.strftime("%A")
td_dates['month'] = td_dates["date"].dt.strftime("%B")
td_dates

,date_id,date,name,month
0,20140401,2014-04-01,Tuesday,April
1011,20140402,2014-04-02,Wednesday,April
2347,20140403,2014-04-03,Thursday,April
3829,20140404,2014-04-04,Friday,April
5656,20140405,2014-04-05,Saturday,April
...,...,...,...,...
3535996,20140926,2014-09-26,Friday,September
3536949,20140927,2014-09-27,Saturday,September
3537954,20140928,2014-09-28,Sunday,September
3538688,20140929,2014-09-29,Monday,September


## TH Viajes

In [ ]:
th_travels = pd.DataFrame(aux)
th_travels.rename(columns = {'base':'base_name','date':'date_id'}, inplace = True)

In [ ]:
th_travels = th_travels.merge(td_latitude, on='lat')
th_travels = th_travels.merge(td_longitude, on='lon')
th_travels = th_travels.merge(td_bases, on='base_name')
th_travels['date_id'] = pd.to_datetime(th_travels['date_id']).dt.strftime("%Y%m%d")

th_travels = th_travels.drop(columns=['lat','lon','day','base_name'])
cols = ['base_code','date_id','hour','lat_id','lon_id']
th_travels = th_travels.reindex(columns=cols)

th_travels

,base_code,date_id,hour,lat_id,lon_id
0,B02512,20140401,0,0,0
1,B02512,20140723,22,29,0
2,B02512,20140721,20,31,0
3,B02512,20140906,16,48,0
4,B02512,20140809,18,70,0
...,...,...,...,...,...
4534322,B02617,20140917,7,6948,11110
4534323,B02617,20140924,5,6965,11168
4534324,B02617,20140924,5,6966,11169
4534325,B02617,20140925,11,6967,11179


## Lectura de credenciales

In [ ]:
with open("cred_uber_bi.json", "r") as f:
  cred=json.load(f)

In [ ]:
host = cred['host']
port = cred['port']
user = cred['username']
pwd = cred['password']
db = cred['db']

In [ ]:
flavor = "mysql"
client = "pymysql"

In [ ]:
conection = create_engine(f"{flavor}+{client}://{user}:{pwd}@{host}:{port}/{db}").connect()

## Preservación de tablas



In [ ]:
td_latitude.to_sql(name="td_latitude", con=conection, index=False, if_exists="replace")

In [ ]:
td_longitude.to_sql(name="td_longitude", con=conection, index=False, if_exists="replace")

In [ ]:
td_bases.to_sql(name="td_bases", con=conection, index=False, if_exists="replace")

In [ ]:
td_dates.to_sql(name="td_dates", con=conection, index=False, if_exists="replace")

In [ ]:
th_travels.to_sql(name="th_travels", con=conection, index=False, if_exists="replace")

In [ ]:
conection.close()